In [1]:
using DataFrames, CSV, DelimitedFiles, JuMP, Gurobi
const GRB_ENV = Gurobi.Env()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x0000000067d37dd0, false, 0)

In [2]:
NUM_CREWS = 10                
BREAK_LENGTH = 2       # how long at base to be considered "rested"

# tradeoffs
BETA = 100             # cost of one area unit burned / cost of mile traveled
ALPHA = 200            # cost of crew-day of suppression / cost of mile traveled
LINE_PER_CREW = 17     # how much perimeter prevented per crew per time period

FIRE_CODE = 1
BASE_CODE = 2

2

In [3]:
struct GlobalData
    
    ff_dist::Matrix{Float64}
    bf_dist::Matrix{Float64}
    ff_tau::Matrix{Int64}
    bf_tau::Matrix{Int64}
    
end

In [4]:
struct RegionData
    
    crew_regions::Vector{Int64}
    fire_regions::Vector{Int64}
    
end

mutable struct RouteData
    
    routes_per_crew::Vector{Int64} # could add in length
    route_costs::Matrix{Real}
    
end

mutable struct SuppressionPlanData
    
    plans_per_fire::Vector{Int64} # could add in length
    plan_costs::Matrix{Real}
    
end

In [49]:
function get_rotation_orders(crew_regions)
    
    # initialize output
    out = Dict()
    
    # get the unique regions where there are crews
    regions = unique(crew_regions)
    
    # for each region
    for region in regions
        
        # initialize dictionary corresponding to the order
        out[region] = Dict() 
        crews_in_region = 0
        
        # for each crew in the region
        for crew in 1:NUM_CREWS
            
            if crew_regions[crew] == region
                
                # update crew count, log rotation order 
                crews_in_region += 1
                out[region][crew] = crews_in_region
            end
        end
    end
    
    return out
end

get_rotation_orders (generic function with 1 method)

In [5]:
# set path to all input data
in_path = "data/processed"

# get inital fire perimeters and no-suppression progression parameters
M = readdlm(in_path * "/sample_growth_patterns.csv", ',')
start_perims = M[:, 1]
progressions = M[:, 2:15]

NUM_TIME_PERIODS = size(M)[2] - 1 
NUM_FIRES = size(M)[1]       

# get distance from fire f to fire g 
fire_dists =  readdlm(in_path * "/fire_distances.csv", ',')

# get distance from base c to fire g (NUM_CREWS-by-NUM_FIRES)
base_fire_dists =  readdlm(in_path * "/base_fire_distances.csv", ',')

# initialize travel times (number of periods) from fire f to fire g
tau = convert(Array{Int}, ones(size(fire_dists)))

# initialize number of periods to travel from base c to fire g (NUM_CREWS-by-NUM_FIRES)
tau_base_to_fire = convert(Array{Int}, ones((size(base_fire_dists))))



global_data = GlobalData(fire_dists, base_fire_dists, tau, tau_base_to_fire)

fire_dists = 0
base_fire_dists = 0
tau = 0
tau_base_to_fire = 0

# read intial crew statuses (location, period by which they must rest)
# (-1 in current_fire means crew is currently at base)
# (rested_periods is the amount of time crew has been at base, relevant for completing rest)
crew_starts = CSV.read(in_path * "/sample_crew_starts.csv", DataFrame)
println(size(crew_starts)[1])
rest_by = crew_starts[!, "rest_by"]
current_fire = crew_starts[!, "current_fire"]
rested_periods = crew_starts[!, "rested_periods"];

10


In [50]:
rd = RegionData([1, 1, 1, 1, 1, 2, 2, 2, 2, 2], [1, 1, 2, 2])
rot_order = get_rotation_orders(rd.crew_regions)

Dict{Any, Any} with 2 entries:
  2 => Dict{Any, Any}(6=>1, 7=>2, 10=>5, 9=>4, 8=>3)
  1 => Dict{Any, Any}(5=>5, 4=>4, 2=>2, 3=>3, 1=>1)

In [7]:
# crew, from_type, from_ix, to_type, to_ix, from_time, to_time, from_rested, to_rested, exited_region

In [7]:
function arc_exits_region(crew, from_type, from_ix, to_type, to_ix, region_data)
    
    # get the region where the arc originates
    from_region = 0
    if from_type == FIRE_CODE
        from_region = region_data.fire_regions[from_ix]
    elseif from_type == BASE_CODE
        from_region = region_data.crew_regions[from_ix]
    else
        throw(DomainError(from_type, "from_type invalid"))
    end
    
    # get the region where the arc terminates
    to_region = 0
    if to_type == FIRE_CODE
        to_region = region_data.fire_regions[to_ix]
    elseif to_type == BASE_CODE
        to_region = region_data.crew_regions[to_ix]
    else
        throw(DomainError(from_type, "to_type invalid"))
    end
    
    # if these are different regions
    if from_region != to_region
        
        # if the crew is leaving its home region
        if region_data.crew_regions[crew] == from_region
        
            # return the region that the arc exited
            return from_region
        
        end
        
    end
    
    # otherwise
    return 0
    
end     

arc_exits_region (generic function with 1 method)

In [8]:
gd = global_data;

In [9]:
# get fire-to-fire arcs
ff = [[c, FIRE_CODE, f_from, FIRE_CODE, f_to, t_from, t_from + gd.ff_tau[f_to, f_from], rest, rest]
      for c=1:NUM_CREWS, f_from=1:NUM_FIRES, f_to=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
ff = copy(reduce(hcat, ff)')

# get fire-to-fire arcs from start, based on current crew locations
from_start_ff = [[c, FIRE_CODE, current_fire[c], FIRE_CODE, f_to, 0, gd.ff_tau[f_to, current_fire[c]], 0, 0]
                  for c=1:NUM_CREWS, f_to=1:NUM_FIRES if current_fire[c] != -1]
from_start_ff = copy(reduce(hcat, from_start_ff)')

# get base-to-fire arcs
rf = [[c, BASE_CODE, c, FIRE_CODE, f_to, t_from, t_from + gd.bf_tau[c, f_to], rest, rest]
       for c=1:NUM_CREWS, f_to=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
rf = copy(reduce(hcat, rf)')

# get base-to-fire arcs from start
from_start_rf = [[c, BASE_CODE, c, FIRE_CODE, f_to, 0, gd.bf_tau[c, f_to], 0, 0]
                  for c=1:NUM_CREWS, f_to=1:NUM_FIRES if current_fire[c] == -1]
from_start_rf = copy(reduce(hcat, from_start_rf)')

# get fire-to-base arcs
fr = [[c, FIRE_CODE, f_from, BASE_CODE, c, t_from, t_from + gd.bf_tau[c, f_from], rest, rest]
       for c=1:NUM_CREWS, f_from=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
fr = copy(reduce(hcat, fr)')

# get fire-to-base arcs from start, based on current crew locations
from_start_fr = [[c, FIRE_CODE, current_fire[c], BASE_CODE, c, 0, gd.bf_tau[c, current_fire[c]], 0, 0]
                  for c=1:NUM_CREWS if current_fire[c] != -1]
from_start_fr = copy(reduce(hcat, from_start_fr)')

# get base-to-base arcs
rr = [[c, BASE_CODE, c, BASE_CODE, c, t_from, t_from + 1 + (BREAK_LENGTH - 1) * rest, 0, rest]
      for c=1:NUM_CREWS, t_from=1:NUM_TIME_PERIODS, rest=0:1]
rr = copy(reduce(hcat, rr)')

# get base-to-base arcs from start, based on current days rested
from_start_rr = [[c, BASE_CODE, c, BASE_CODE, c, 0, 
                  1 + (BREAK_LENGTH - max(rested_periods[c], 0) - 1) * rest, 0, rest] 
                  for c=1:NUM_CREWS, rest=0:1 if current_fire[c] == -1]
from_start_rr = copy(reduce(hcat, from_start_rr)')

A = vcat(ff, from_start_ff, rf, from_start_rf, fr, from_start_fr, rr, from_start_rr)

out_of_region = [arc_exits_region(A[i, 1], A[i, 2], A[i, 3], A[i, 4], A[i, 5], rd) 
                 for i in 1:length(A[:, 1])]
A = hcat(A, out_of_region);

In [11]:
function get_distance(from_type, from_ix, to_type, to_ix, fire_fire, base_fire)
    
    dist = 0
    
    # if fire to fire
    if from_type == FIRE_CODE & to_type == FIRE_CODE
        dist = fire_fire[from_ix, to_ix]
    
    # if fire to base
    elseif from_type == FIRE_CODE & to_type == BASE_CODE
        dist = base_fire[to_ix, from_ix]
    
    # if base to fire
    elseif from_type == BASE_CODE & to_type == FIRE_CODE
        dist = base_fire[from_ix, to_ix]
        
    # otherwise dist still 0
    end
    
    return dist
end 

get_distance (generic function with 1 method)

In [12]:
function get_arc_costs(gd, arc_matrix, cost_param_dict)
    
    # initialize costs to 0
    costs = zeros(length(arc_matrix[:, 1]))
    
    # if there is travel cost per mile
    if "cost_per_mile" in keys(cost_param_dict)
        
        # find the miles for each arc
        miles_per_arc =  [get_distance(arc_matrix[i, 2], arc_matrix[i, 3], 
                                       arc_matrix[i, 4], arc_matrix[i, 5], 
                                       gd.ff_dist, gd.bf_dist) for i in 1:length(arc_matrix[:, 1])]
        # add to costs
        costs = costs .+ (cost_param_dict["cost_per_mile"] * miles_per_arc)
    end
    
    # if there are rest violations
    if "rest_violation" in keys(cost_param_dict)
        
        # find the rest violation scores
        rest_violation_matrix = cost_param_dict["rest_violation"]
        rest_violations = [(arc_matrix[i, 8] == 0) & (arc_matrix[i, 6] > 0) ? 
                           rest_violation_matrix[arc_matrix[i, 1], arc_matrix[i, 6]] : 0
                           for i in 1:length(arc_matrix[:, 1])]
        
        # add to costs
        costs = costs .+ rest_violations
    end
    
    if "fight_fire" in keys(cost_param_dict)
        costs = costs .+ [(arc_matrix[i, 4] == FIRE_CODE) ? cost_param_dict["fight_fire"] : 0
                          for i in 1:length(arc_matrix[:, 1])]
    end
    
    return costs
end

get_arc_costs (generic function with 1 method)

In [13]:
function positive(x)
    
    if x > 0
        return 1
    end
    
    return 0
end

function is_one(x)
    
    if x == 1
        return 1
    end
    
    return 0
end

is_one (generic function with 1 method)

In [14]:
# should return matrix indexed by crew, time, 
function get_rest_penalties(rest_by_periods, lambda, accounting_func)
    
    penalties = zeros(NUM_CREWS, NUM_TIME_PERIODS)
    
    for c in 1:NUM_CREWS
        penalties[c, :] = [lambda * accounting_func(t - rest_by_periods[c]) 
                           for t in 1:NUM_TIME_PERIODS]
    end
    
    return penalties    
end

get_rest_penalties (generic function with 1 method)

In [15]:
rest_pen = get_rest_penalties(rest_by, 99999, positive);

In [16]:
cost_params = Dict("cost_per_mile"=> 1, "rest_violation" => rest_pen, "fight_fire" => ALPHA)
arc_costs = get_arc_costs(global_data, A, cost_params);

In [17]:
struct KeyArcIndices
    
    # fire flow data
    f_out::Array{Vector{Int64}}
    f_in::Array{Vector{Int64}}
    
    # base flow data
    b_out::Array{Vector{Int64}}
    b_in::Array{Vector{Int64}}
    
    # total crews suppressing each fire
    supp_fire::Array{Vector{Int64}}
    
    # start constraints
    start::Array{Vector{Int64}}
    
    # assignments out of region
    out_of_region::Array{Vector{Int64}}
    
end 

In [58]:
function define_network_constraint_data(arcs)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # get number of arcs
    n_arcs = length(arcs[:, 1])
      
    ## flow balance ##
    
    # initialize arrays of vectors for flow balance
    f_out = Array{Vector{Int64}}(undef, C, G, T, 2)
    f_in = Array{Vector{Int64}}(undef, C, G, T, 2)
    b_out = Array{Vector{Int64}}(undef, C, T, 2)
    b_in = Array{Vector{Int64}}(undef, C, T, 2)
    start = Array{Vector{Int64}}(undef, C)
    out_of_region = Array{Vector{Int64}}(undef, C, T+1)
    
    # for each crew
    for crew in 1:C
        
        # get indices of this crew's arcs only
        crew_ixs = [i for i in 1:n_arcs if arcs[i, 1] == crew]
        
        # get time 0 indices
        start[crew] = [i for i in crew_ixs if arcs[i, 6] == 0]
        
        # for each time period (including start)
        for tm in 0:T
        
            # get indices for out of region assignments
            out_of_region[crew, tm+1] = [i for i in crew_ixs if
                                           (arcs[i, 6] == tm) &
                                           (arcs[i, 10] > 0)
                                        ]
        end
        
        # for each time period
        for tm in 1:T
            
            # for each rest state
            for rest in 1:2
                
                # get arcs leaving crew base at this time with this rest
                b_out[crew, tm, rest] = [i for i in crew_ixs if
                                         (arcs[i, 2] == BASE_CODE) &
                                         (arcs[i, 6] == tm) &
                                         (arcs[i, 8] == rest-1)
                                        ]
                
                # get arcs entering crew base at this time with this rest
                b_in[crew, tm, rest] = [i for i in crew_ixs if
                                        (arcs[i, 4] == BASE_CODE) &
                                        (arcs[i, 7] == tm) &
                                        (arcs[i, 9] == rest-1)
                                       ]
                # for each fire
                for fire in 1:G
                    
                    # get arcs where this crew leaves this fire at this time
                    # with this rest state
                    f_out[crew, fire, tm, rest] = [i for i in crew_ixs if
                                                   (arcs[i, 2] == FIRE_CODE) &
                                                   (arcs[i, 3] == fire) &
                                                   (arcs[i, 6] == tm) &
                                                   (arcs[i, 8] == rest-1)
                                                   ]
                    
                    # get arcs where this crew enters this fire at this time
                    # with this rest state
                    f_in[crew, fire, tm, rest] = [i for i in crew_ixs if
                                                  (arcs[i, 4] == FIRE_CODE) &
                                                  (arcs[i, 5] == fire) &
                                                  (arcs[i, 7] == tm) &
                                                  (arcs[i, 9] == rest-1)
                                                  ]
                end
            end
        end
    end
    
    ## linking constraints ##
    linking = Array{Vector{Int64}}(undef, G, T)
    for fire in 1:G
        for tm in 1:T
            
            # we count the crew as working *where they arrived* during this timestep
            linking[fire, tm] = [i for i in 1:n_arcs if (arcs[i, 4] == FIRE_CODE) &
                                                        (arcs[i, 5] == fire) &
                                                        (arcs[i, 7] == tm)]
        end
    end
    
    
    return KeyArcIndices(f_out, f_in, b_out, b_in, linking, start, out_of_region)
end

define_network_constraint_data (generic function with 1 method)

In [56]:
[i for i in keys(rot_order[1])]

5-element Vector{Int64}:
 5
 4
 2
 3
 1

In [ ]:
    @constraint(m, out_of_region[c=1:C, t=0:T],
    
        # out of region penalty is at least
        q[c, t] >=
        
            # this crew's cumulative rotations
            sum(z[constraint_data.out_of_region[c, t_0+1]] for t_0=0:t) 
        
        - 
        
            # average cumulative rotations among all crews in same region
            sum(z[constraint_data.out_of_region[c_0, t_0+1]] 
                for c_0 in keys(rotation_order[regs[c]]), t_0=0:t) /
            length(keys(rotation_order[regs]))
        
        -
        
            # normalizing factor for specific crew rotation order
            (1 - rotation_order[regs[c]][c] / length(keys(rotation_order[regs[c]])))
        
        -
            # big-M for if crew goes not leave region at this time
            T * (1 - sum(z[constraint_data.out_of_region[c, t+1]]))
        
    )


In [108]:
function full_formulation(integer_routes, region_data, constraint_data, rotation_order, 
                          costs, progs, perims, beta, gamma)
    
    # get number of arcs
    n_arcs = length(costs)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    regs = region_data.crew_regions
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))

    # fire suppression plan section
    @variable(m, p[g=1:G, t=1:T+1] >= 0)
    @variable(m, l[g=1:G, t=1:T] >= 0)
    @constraint(m, perim_growth[g=1:G, t=1:T], p[g, t+1] >= progs[g, t] * 
                                                           (p[g, t] - l[g, t] / 2) - l[g, t] / 2)
    @constraint(m, perim_start[g=1:G], p[g, 1] == perims[g])

    # routing plan section
    if integer_routes
        @variable(m, z[1:n_arcs] >= 0, Int)
    else
        @variable(m, z[1:n_arcs] >= 0)
    end
    
    @variable(m, q[1:C, 0:T] >= 0, Int)
    
    @constraint(m, out_of_region[c=1:C, t=0:T],
    
        # out of region penalty is at least
        q[c, t] >=
        
            # this crew's cumulative rotations
            sum(z[i] for t_0=0:t, i in constraint_data.out_of_region[c, t_0+1]) 
        
        - 
        
            # average cumulative rotations among all crews in same region
            sum(z[i] for c_0 in keys(rotation_order[regs[c]]), t_0=0:t, 
                i in constraint_data.out_of_region[c_0, t_0+1]) /
            length(keys(rotation_order[regs[c]]))
        
        -
        
            # normalizing factor for specific crew rotation order
            (1 - rotation_order[regs[c]][c] / length(keys(rotation_order[regs[c]])))
        
        -
            # big-M for if crew goes not leave region at this time
            T * (1 - sum(z[i] for i in constraint_data.out_of_region[c, t+1]))
        
    )


    @constraint(m, fire_flow[c=1:C, g=1:G, t=1:T, rest=1:2],

            sum(z[constraint_data.f_out[c, g, t, rest]]) ==
            sum(z[constraint_data.f_in[c, g, t, rest]])
    
    )
    
    @constraint(m, base_flow[c=1:C, t=1:T, rest=1:2],

            sum(z[constraint_data.b_out[c, t, rest]]) ==
            sum(z[constraint_data.b_in[c, t, rest]])
    
    )


    @constraint(m, linking[g=1:G, t=1:T],

        sum(z[constraint_data.supp_fire[g, t]]) >= l[g, t] / LINE_PER_CREW
    )

    # build start constraint
    @constraint(m, start[c=1:C], 

        sum(z[constraint_data.start[c]]) == 1
    )
    
    # build out_of_region constraints
    

    @objective(m, Min, 
        beta * (sum(p) - sum(p[1:G, 1])/2 - sum(p[1:G, T+1])/2) + 
        sum(z .* costs) + sum(q) * gamma
    )
    
    return m, p, l, z, q, out_of_region
    
end

full_formulation (generic function with 2 methods)

In [109]:
c_data = define_network_constraint_data(A);

In [110]:
m, p, l, z, q, oor = full_formulation(true, rd, c_data, rot_order, arc_costs, 
                              progressions, start_perims, BETA, 1000);

In [117]:
A[[i for i=1:length(A[:, 1]) if (value(z[i]) > 0.5) & (A[i, 10] > 0)], :]

13×10 Matrix{Int64}:
  6  1   3  1  2   1   2  0  0  2
  7  1   3  1  2   1   2  0  0  2
  8  1   3  1  2   1   2  0  0  2
  9  1   3  1  2   2   3  0  0  2
  3  1   1  1  3   0   1  0  0  1
  4  1   1  1  3   0   1  0  0  1
 10  2  10  1  2   3   4  1  1  2
  9  2   9  1  1   6   7  1  1  2
  7  2   7  1  1   7   8  1  1  2
  8  2   8  1  1   7   8  1  1  2
  6  2   6  1  1  10  11  1  1  2
  1  2   1  1  3   0   1  0  0  1
  2  2   2  1  3   0   1  0  0  1

In [127]:
function init_route_subproblem(crew_ixs, crew, constraint_data, integer_routes=false)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))

    # routing plan section
    if integer_routes
        @variable(m, z[crew_ixs] >= 0, Int)
    else
        @variable(m, z[crew_ixs] >= 0)
    end


    @constraint(m, fire_flow[g=1:G, t=1:T, rest=1:2],

            sum(z[constraint_data.f_out[crew, g, t, rest]]) ==
            sum(z[constraint_data.f_in[crew, g, t, rest]])
    
    )
    
    @constraint(m, base_flow[t=1:T, rest=1:2],

            sum(z[constraint_data.b_out[crew, t, rest]]) ==
            sum(z[constraint_data.b_in[crew, t, rest]])
    
    )

    # build start constraint
    @constraint(m, start, 

        sum(z[constraint_data.start[crew]]) == 1
    )
    
    return Dict("m" => m, "z" => z, "ff" => fire_flow)
    
end

init_route_subproblem (generic function with 2 methods)

In [130]:
crew = 1
ixs = [i for i in 1:length(A[:, 1]) if A[i, 1] == crew]
d = init_route_subproblem(ixs, crew, c_data);

In [132]:
d["ff"][1, 1, 1]

fire_flow[1,1,1] : z[1] + z[31] + z[61] - z[3379] + z[3391] == 0.0

In [133]:
A[[1, 31, 61, 3379, 3391], :]

5×10 Matrix{Int64}:
 1  1  1  1  1  1  2  0  0  0
 1  1  1  1  2  1  2  0  0  0
 1  1  1  1  3  1  2  0  0  1
 1  2  1  1  1  0  1  0  0  0
 1  1  1  2  1  1  2  0  0  0

In [122]:
value(q[9, 6])

1.0

In [111]:
optimize!(m)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1367 rows, 4761 columns and 49989 nonzeros
Model fingerprint: 0xbfb53cd0
Variable types: 87 continuous, 4674 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-02, 1e+01]
  Objective range  [5e+01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 201 rows and 683 columns
Presolve time: 0.34s
Presolved: 1166 rows, 4078 columns, 43469 nonzeros
Variable types: 14 continuous, 4064 integer (353 binary)

Root relaxation: objective 1.141712e+06, 1637 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1141711.79    0   31          - 1141711.79      -     -    0s
     0     0 1146563.52    0   45          - 1146563.52      -     -    1s
     0     0 1146611.24    0   12          - 1146611.24      -     -    1s
     0     0 11

In [106]:
value.(l) / LINE_PER_CREW

3×14 Matrix{Float64}:
 0.0  0.0  0.0  0.0  3.0  7.0  8.0  8.0  8.0  5.0       3.0  2.0  2.0  2.0
 0.0  7.0  9.0  9.0  4.0  0.0  0.0  0.0  0.0  0.999999  0.0  0.0  1.0  0.0
 9.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0  3.0  0.0  5.0

In [34]:
value.(l) / LINE_PER_CREW

3×14 Matrix{Float64}:
 0.0  0.0  0.0  0.0  3.0  7.0  8.0  8.0  8.0  6.0  2.0  2.0  2.0  6.0
 0.0  7.0  9.0  9.0  4.0  0.0  0.0  0.0  0.0  0.0  2.0  1.0  2.0  0.0
 9.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0

In [19]:
A2 = copy(A)
A = 0

0

In [20]:
A2

4524×10 Matrix{Int64}:
  1  1   1  1   1   1   2  0  0  0
  2  1   1  1   1   1   2  0  0  0
  3  1   1  1   1   1   2  0  0  0
  4  1   1  1   1   1   2  0  0  0
  5  1   1  1   1   1   2  0  0  0
  6  1   1  1   1   1   2  0  0  0
  7  1   1  1   1   1   2  0  0  0
  8  1   1  1   1   1   2  0  0  0
  9  1   1  1   1   1   2  0  0  0
 10  1   1  1   1   1   2  0  0  0
  1  1   2  1   1   1   2  0  0  0
  2  1   2  1   1   1   2  0  0  0
  3  1   2  1   1   1   2  0  0  0
  ⋮                 ⋮            
  7  2   7  2   7  14  16  0  1  0
  8  2   8  2   8  14  16  0  1  0
  9  2   9  2   9  14  16  0  1  0
 10  2  10  2  10  14  16  0  1  0
  1  2   1  2   1   0   1  0  0  0
  2  2   2  2   2   0   1  0  0  0
  6  2   6  2   6   0   1  0  0  0
  9  2   9  2   9   0   1  0  0  0
  1  2   1  2   1   0   2  0  1  0
  2  2   2  2   2   0   2  0  1  0
  6  2   6  2   6   0   2  0  1  0
  9  2   9  2   9   0   1  0  1  0

In [33]:

c_data.f_out[3, 3, 3, 1]

4-element Vector{Int64}:
  203
  233
  263
 3473

In [24]:
a.start[1]

5-element Vector{Int64}:
 3379
 3383
 3387
 4517
 4521

In [39]:
A[c_data.out_of_region[1, 9], :]

6×10 Matrix{Int64}:
 1  1  1  1  3  9  10  0  0  1
 1  1  2  1  3  9  10  0  0  1
 1  1  1  1  3  9  10  1  1  1
 1  1  2  1  3  9  10  1  1  1
 1  2  1  1  3  9  10  0  0  1
 1  2  1  1  3  9  10  1  1  1

In [17]:
A2[9:14, :]

6×10 Matrix{Int64}:
  9  1  1  1  1  1  2  0  0  0
 10  1  1  1  1  1  2  0  0  0
  1  1  2  1  1  1  2  0  0  0
  2  1  2  1  1  1  2  0  0  0
  3  1  2  1  1  1  2  0  0  0
  4  1  2  1  1  1  2  0  0  0

In [33]:
(sum(get_arc_costs(global_data, A2, Dict("cost_per_mile"=>10, "rest_violation" => ones(NUM_CREWS, NUM_TIME_PERIODS)))) - 
    sum(get_arc_costs(global_data, A2, Dict("cost_per_mile"=>10))))

2380.0

In [41]:
sum(1 .- A2[:, 8]) - sum(A2[:, 6] .== 0)

2380

In [32]:
d = Dict(3=>'a')

Dict{Int64, Char} with 1 entry:
  3 => 'a'

In [34]:
4 in keys(d)

false

In [8]:
struct ArcData
    
# crew, from_type, from_ix, to_type, to_ix, from_time, to_time, from_rested, to_rested, exited_region

LoadError: syntax: incomplete: premature end of input

In [9]:
function get_out_of_region_stats(region, arcs_used, region_data)
    """
    """
    
    # restrict to the arcs that exited the given region
    out_of_region_ixs = [i for i in length(arcs_used[:, 1]) if arcs_used[i, 10] == region]
    out_of_region_arcs = arcs_used[out_of_region_ixs, :]
    
    # get the crews associated with this region
    crews = [i for i in 1:length(region_data.crew_regions) if region_data.crew_regions[i] == region]
    
    # initialize output array of indicator variables for crews exiting region
    out_array = zeros(length(crews), NUM_TIME_PERIODS)
    
    # for each crew in the region
    for i in 1:length(crews)
        
        # restrict to the arcs involving this crew
        crew_ixs = [j for j in length(out_of_region_arcs[:, 1]) if arcs_used[j, 1] == crews[i]]
        crew_arcs = out_of_region_arcs[crew_ixs, :]
        
        # get the times of rotation
        rotation_times = [t in crew_arcs[:, 6] ? 1 : 0 for t in 1:NUM_TIME_PERIODS]
        
        # update output for crew
        out_array[i, :] = rotation_times
        
    end
    
    return out_array
end

LoadError: syntax: incomplete: "function" at In[9]:1 requires end